How to find Tuesdays late in the month?

In [4]:
function getTuesdays(month, year) {
    var d = new Date(year, month, 1),
        tuesdays = [];

    d.setDate(d.getDate() + (9 - d.getDay()) % 7)
    while (d.getMonth() === month) {
        tuesdays.push(new Date(d.getTime()));
        d.setDate(d.getDate() + 7);
    }

    return tuesdays;
}

var month = (new Date()).getMonth();
var results = [];
for(var m = month; month <= 12; month++) {
    results = results.concat(getTuesdays(month, 2017).filter((d) => d.getDate() > 25));
}
console.log(results.map(d => d.getDate() + '/' + (d.getMonth()+1)).join('\r\n'))

29/8
26/9
31/10
28/11
26/12


Convert a date to ISO?

In [5]:
var ISODateString = (d) => {
    function pad(n){return n<10 ? '0'+n : n}
    return d.getUTCFullYear()+'-'
        + pad(d.getUTCMonth()+1)+'-'
        + pad(d.getUTCDate())+'T'
        + pad(d.getUTCHours())+':'
        + pad(d.getUTCMinutes())+':'
        + pad(d.getUTCSeconds())+'-00:00';
};
(ISODateString);

[Function: ISODateString]

How to sum a list of events?



In [6]:
// sum up events
var total = 0;
resultEvents.forEach(e => {
    if(typeof e.event.end === 'undefined') {
        console.log(e.event);
    }
    total += new Date(e.event.end.dateTime).getTime() 
        - new Date(e.event.start.dateTime).getTime();
});

console.log(total / 1000 / 60 / 60);

53


Number of days between events?

In [ ]:
// subtract the events from the previous one
var contributing = [];
resultEvents.sort((a, b) => a.start - b.start);
resultEvents.forEach((e, i) => {
    if(i == 0) {
        return;
    }
    var diff = (e.start.getTime() - resultEvents[i - 1].start.getTime()) / 1000 / 60 / 60 / 24;
    // if it is greater than 1 months it can't help our graph
    //if(diff < 30 && diff > 0) {
        contributing[contributing.length] = {days: diff, start: e.start, event: e.event, i: contributing.length};
    //}
});


How to display recurrence line graph?


In [25]:
var execSync = require('child_process').execSync;
try {
    require.resolve('d3-node');
} catch (e) {
    execSync('npm install d3-node');
}


var D3Node = require('d3-node');
var d3n = new D3Node(); // initializes D3 with container element 
var d3 = d3n.d3;

// set the dimensions and margins of the graph
var margin = {top: 20, right: 20, bottom: 30, left: 50},
    width = 960 - margin.left - margin.right,
    height = 500 - margin.top - margin.bottom;

// set the ranges
var x = d3.scaleTime().range([0, width]);
var y = d3.scaleLinear().range([height, 0]);

// define the line
var valueline = d3.line()
    .x(function(d) { return x(d.start); })
    .y(function(d) { return y(d.days); });

// append the svg obgect to the body of the page
// appends a 'group' element to 'svg'
// moves the 'group' element to the top left margin
var svg = d3n.createSVG(width + margin.left + margin.right,
                        height + margin.top + margin.bottom)
  .append("g")
    .attr("transform",
          "translate(" + margin.left + "," + margin.top + ")");

// Get the data


// Scale the range of the data
x.domain(d3.extent(contributing, function(d) { return d.start; }));
y.domain([0, d3.max(contributing, function(d) { return d.days; })]);

// Add the valueline path.
svg.append("path")
  .data([contributing])
  .attr("class", "line")
    .attr("fill", "none")
    .attr("stroke", "#000")
  .attr("d", valueline);

// Add the X Axis
svg.append("g")
  .attr("transform", "translate(0," + height + ")")
  .call(d3.axisBottom(x));

// Add the Y Axis
svg.append("g")
  .call(d3.axisLeft(y));

$$.svg(d3n.svgString());